In [1]:
from numpy import vstack
from numpy import sqrt
import pandas as pd
from sklearn.metrics import mean_squared_error
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import MSELoss
from torch.nn.init import xavier_uniform_

In [10]:
# dataset definition
class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, path):
        # load the csv file as a dataframe
        df = pd.read_csv(path, header=None)
        # store the inputs and outputs
        self.features = df.values[:, :-1].astype('float32')
        self.target = df.values[:, -1].astype('float32')
        # ensure target has the right shape
        self.target = self.target.reshape((len(self.target), 1))
 
    # number of rows in the dataset
    def __len__(self):
        return len(self.features)
 
    # get a row at an index
    def __getitem__(self, idx):
        return [self.features[idx], self.target[idx]]
 
    # get indexes for train and test rows
    def get_splits(self, n_test=0.33):
        # determine sizes
        test_size = round(n_test * len(self.features))
        train_size = len(self.features) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])

In [3]:
# model definition
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, 10)
        xavier_uniform_(self.hidden1.weight)
        self.act1 = Sigmoid()
        # second hidden layer
        self.hidden2 = Linear(10, 8)
        xavier_uniform_(self.hidden2.weight)
        self.act2 = Sigmoid()
        # third hidden layer and output
        self.hidden3 = Linear(8, 1)
        xavier_uniform_(self.hidden3.weight)
 
    # forward propagate input
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
         # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # third hidden layer and output
        X = self.hidden3(X)
        return X

In [4]:
# prepare the dataset
def prepare_data(path):
    # load the dataset
    dataset = CSVDataset(path)
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    return train_dl, test_dl

In [5]:
# train the model
def train_model(train_dl, model):
    # define the optimization
    criterion = MSELoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    # enumerate epochs
    for epoch in range(100):
        # enumerate mini batches
        for i, (inputs, targets) in enumerate(train_dl):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            pred = model(inputs)
            # calculate loss
            loss = criterion(pred, targets)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()

In [6]:
# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        pred = model(inputs)
        # retrieve numpy array
        pred = pred.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))
        # store
        predictions.append(pred)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate mse
    mse = mean_squared_error(actuals, predictions)
    return mse

In [7]:
# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    pred = model(row)
    # retrieve numpy array
    pred = pred.detach().numpy()
    return pred

In [8]:
path = 'D://DS//Stater_Projects//housing.txt'

In [11]:
train_dl, test_dl = prepare_data(path)

In [12]:
print(len(train_dl.dataset), len(test_dl.dataset))

339 167


In [13]:
model = MLP(13)

In [14]:
train_model(train_dl, model)

In [15]:
mse = evaluate_model(test_dl, model)

In [16]:
print('MSE: %.3f, RMSE: %.3f' % (mse, sqrt(mse)))

MSE: 64.876, RMSE: 8.055


In [17]:
trial_pred = [0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,396.90,4.98]
pred = predict(trial_pred, model)
print('Predicted: %.3f' % pred)

Predicted: 23.078
